# Import libs and utils

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import json
from pathlib import Path

DATA_DIR = '/kaggle/input/arc-prize-2025/'
data = {}
with open(Path(DATA_DIR) / 'arc-agi_training_challenges.json') as f:
    train_challenges = json.load(f)
with open(Path(DATA_DIR) / 'arc-agi_training_solutions.json') as f:
    train_solutions = json.load(f)

with open(Path(DATA_DIR) / 'arc-agi_evaluation_challenges.json') as f:
    eval_challenges = json.load(f)
with open(Path(DATA_DIR) / 'arc-agi_evaluation_solutions.json') as f:
    eval_solutions = json.load(f)

with open(Path(DATA_DIR) / 'arc-agi_test_challenges.json') as f:
    test_challenges = json.load(f)

print(f"Training tasks: {len(train_challenges)}")
print(f"Training solutions: {len(train_solutions)}")
print(f"Evaluation tasks: {len(eval_challenges)}")
print(f"Evaluation solutions: {len(eval_solutions)}")
print(f"Test tasks: {len(test_challenges)}")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session# Utils
import random
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torchsummary import summary

def start_train(train_func, *args, **kwargs):
    for epoch in range(50):
        running_loss = 0.0
        print("epoch", epoch+1)
        i = 0
        train_func(*args, **kwargs)
# load data
import pandas as pd
for i in range(0, 1):
    key = list(train_challenges.keys())[i]
    print(train_challenges[key])
    print("----train----")
    print(train_solutions[key])
    print("----train----")

# Discriminator

## data_1: data from ARC

In [ ]:
## prepare train data
inputs = []
labels = []
discriminator_train_data = []
discriminator_test_data = []
# get the inputs; data is a list of [inputs, labels]
for task_name, task_data in train_challenges.items():
    task_big_bag = [task_name, []]
    task_bag = [[], []]
    for temp in task_data["train"]:
        task_bag[0].append(temp["input"])
        task_bag[1].append(temp["output"])
    for ii in task_bag[0]:
        temp = [task_bag[0], task_bag[1], ii, 0]
        task_big_bag[1].append(temp)
    for ii in task_bag[1]:
        temp = [task_bag[0], task_bag[1], ii, 1]
        task_big_bag[1].append(temp)

    discriminator_train_data.append(task_big_bag)
print(len(discriminator_train_data))
print(len(discriminator_train_data[0]))
print(len(discriminator_train_data[0][1]))
list(train_challenges.items())[0][1]["train"]
# train using permutation (asume permutation will preserve correlation)

        

In [ ]:
test = [[[1, 2], [3, 4]]]
test_1 = torch.stack([torch.stack([torch.Tensor(t) == i for i in range(0, 10)]).float() for t in test]) 
print(test_1.shape)
test_2 = [torch.roll(torch.Tensor(t), i, 1) for i in range(0, 10) for t in [torch.stack([torch.Tensor(t) == i for i in range(0, 10)]).float() for t in test]]
test_2 = torch.stack([torch.roll(torch.Tensor(t), i, 1) for i in range(0, 10) for t in [torch.stack([torch.Tensor(t) == i for i in range(0, 10)]).float() for t in test]])
print(test_2.shape)

In [ ]:
import torch
t1 = torch.ones((3, 4)).squeeze()
t2 = torch.zeros((3, 4)).squeeze()
t3 = torch.stack([t1, t2], dim=1)
print(t3.shape)
t4 = t3.view(1, t3.shape[1]*t3.shape[0], -1)
print(t4)

t5 = nn.Conv2d(1, 256, kernel_size=(2, 1), stride=(2, 1))(t4)
print(t5.shape)
t6 = nn.AdaptiveAvgPool2d((1))(t5)
print(t6.shape)

## model

In [ ]:
# create a pretrain model for discriminator
# do 2 things: learn how to pick up pattern from inputs and outputs
# 
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


class DynamicLinear(nn.Linear):
    def __init__(self, weights=None, bias=None, trainable=True, *arg, **kwargs):
        super().__init__(*arg, **kwargs)
        with torch.no_grad():
            self.weight.copy_(weights)
            self.bias.copy_(bias)
        if not trainable:
            for param in self.parameters():
                param.requires_grad = False

# ---------------------------------------- #   
class BatchMatMulLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, weight, bias):
        # x: (batch, in_dim)
        # weight: (batch, out_dim, in_dim)
        # bias: (batch, out_dim)
        out = torch.bmm(weight, x.unsqueeze(-1)).squeeze(-1) + bias
        return out

# ---------------------------------------- #
class HyperEncoder(nn.Module):
    def __init__(self, in_features, out_features, weight, bias, filters:torch.Tensor):
        super().__init__()
        
        # CNN
       
        filters = filters
        F.conv2d(inputs, filters, padding=1)

        
        # Define learnable weights and bias as nn.Parameter
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

        # Initialize parameters (optional, but good practice)
        nn.init.kaiming_uniform_(self.weight, nonlinearity='relu')
        nn.init.zeros_(self.bias)

    def forward(self, x):
        # Perform matrix multiplication and add bias
        x = torch.matmul(x, self.weight.T) + self.bias
        # Apply ReLU activation
        x = torch.relu(x)
        return x

# ---------------------------------------- #
class DynamicLinearGenerator(nn.Module):
    def __init__(self, input_dim, params_dim, target_in_dim, target_out_dim):
        super().__init__()
        self.input_dim = input_dim
        self.target_in_dim = target_in_dim
        self.target_out_dim = target_out_dim

        self.filter_embedding = nn.Embedding()

        self.
        # self.multihead_attention = nn.MultiheadAttention(Q, K, V)
        
        self.ffn_block = nn.Sequential(
            nn.Linear(input_dim, params_dim, bias=True),
            nn.ReLU(),
            nn.Linear(params_dim, target_in_dim * target_out_dim + target_out_dim, bias=True),
        )
        
        
        # Q,K,V will be Q = Task Input, K = Task Output, V = F(Input, Output)

    
    def forward(self, cross_conv, x_conv, y_conv,fit_data=None):
        # Generate flat parameters
        params = self.ffn_block(cross_conv).squeeze()
        
        # Split into weight and bias
        weight = params[:self.target_in_dim * self.target_out_dim]
        bias = torch.Tensor(params[self.target_in_dim * self.target_out_dim:]).squeeze()
        
        # Reshape weight into (out_dim, in_dim) for linear layer
        weight = torch.Tensor(weight.view(-1, self.target_out_dim, self.target_in_dim)).squeeze()
        print(weight.shape)
        dynamic_linear = DynamicLinear(weight, bias, in_features=self.target_in_dim, out_features=self.target_out_dim)
        
        # fit dynamic linear

        self.fit(dynamic_linear, fit_data[0], fit_data[1])

        return dynamic_linear

    def fit(self, model, x_input, y_label, epochs=10, tloss= 0.5):
        # SVM loss
        criterion = torch.nn.HingeEmbeddingLoss(margin=1.0, size_average=None, reduce=None, reduction='mean')
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        # calucate loss
        avg_loss = 0.0
        for i in x_input:
            print(i.shape)
            optimizer.zero_grad()
            out = model(i)
            loss = criterion(x_out, y_label)
            loss.backward()
            loss_ += loss.item()
        
        # train
        for epoch in range(epochs):
            for i in x_input:
                optimizer.zero_grad()
                out = model(i)
                loss = criterion(x_out, y_label)
                loss.backward()
                optimizer.step()



    
class Discriminator(nn.Module):
    """
        Discriminate if an image is a task output or not
        Must be fast train for each task
        Input: 
    """
    def __init__(self, img_channels=10, learning_block_dim=128):
        super(Discriminator, self).__init__()
        # self.block1 = ParallelConvBlock(img_channels, 32)
        # self.block2 = ParallelConvBlock(32, 64)
        self.learning_block=128
        self.out_channels = 64

        self.cnn_block_s = nn.Sequential(
            nn.Conv2d(img_channels, 128, kernel_size=(3, 3), padding=2),
            nn.Conv2d(128, self.out_channels, kernel_size=(3, 3), padding=2),
            nn.BatchNorm2d(self.out_channels),
            nn.AdaptiveAvgPool2d((1))
        )

        self.cnn_block_w = nn.Sequential(
            nn.Conv2d(img_channels, 128, kernel_size=(1, 3), padding=2),
            nn.Conv2d(128, self.out_channels, kernel_size=(3, 3), padding=2),
            nn.BatchNorm2d(self.out_channels),
            nn.AdaptiveAvgPool2d((1))

        )

        self.cnn_block_h = nn.Sequential(
            nn.Conv2d(img_channels, 128, kernel_size=(3, 1), padding=2),
            nn.Conv2d(128, self.out_channels, kernel_size=(3, 3), padding=2),
            nn.BatchNorm2d(self.out_channels),
            nn.AdaptiveAvgPool2d((1))
        )

        self.cnn_block_cross = nn.Sequential(
            nn.Conv2d(1, 256, kernel_size=(1, 2), stride=(1, 2)),
            # nn.Conv2d(256, 256, kernel_size=(3, 1), stride(), ),
            nn.AdaptiveAvgPool2d((1))
        )

        # GENERATOR
        self.hyper_net = DynamicLinearGenerator(input_dim=256, params_dim=512, target_in_dim=learning_block_dim, target_out_dim=1)


    def forward(self, train_task_inputs, train_task_outputs, target, learning_effort=10):
        """
            Train
        """
        # x = torch.stack([train_task_inputs == i for i in range(0, 10)]).float().unsqueeze(0) # decompose in into channels
        # y = torch.stack([train_task_inputs == i for i in range(0, 10)]).float().unsqueeze(0) # decompose in into channels
        # learning all the permutation of an input and an output
        x = torch.stack([torch.roll(torch.Tensor(t), i, 1) for i in range(0, 10) for t in [torch.stack([torch.Tensor(t) == i for i in range(0, 10)]).float() for t in train_task_inputs]])
        y = torch.stack([torch.roll(torch.Tensor(t), i, 1) for i in range(0, 10) for t in [torch.stack([torch.Tensor(t) == i for i in range(0, 10)]).float() for t in train_task_outputs]])
        # sampling
        # input features [N, 64]
        x_s = self.cnn_block_s(x).squeeze()
        x_h = self.cnn_block_h(x).squeeze()
        x_v = self.cnn_block_w(x).squeeze()
        
        x_sh_concat = torch.concat([x_s, x_h, x_v], dim=1) # [N, 128]

        # output features [N, 32]
        y_s = self.cnn_block_s(y).squeeze()
        y_h = self.cnn_block_h(y).squeeze() 
        y_v = self.cnn_block_w(y).squeeze()

        y_sh_concat = torch.concat([y_s, y_h, y_v], dim=1) # [N, 128]
        print("x_sh_concat", x_sh_concat.shape)

        in_out_concat = torch.stack([x_sh_concat, y_sh_concat], dim=1)
        in_out_concat = in_out_concat.view(1, in_out_concat.shape[1]*in_out_concat.shape[0], -1)
        in_out_pair = self.cnn_block_cross(in_out_concat)
        
        dynamic_linear = self.hyper_net(in_out_pair.squeeze()) # [N, 192]
        
        out = dynamic_linear(target)
        
        # task_relation, task_input_relation, task_output_relation = dynamic_linear()
        
        return torch.nn.Sigmoid(out)

            
        
discriminator = Discriminator(10)
try:
    discriminator.load_state_dict(torch.load(f'/kaggle/working/{discriminator.__class__.__name__}.pth', weights_only=True))
except FileNotFoundError as e:
    display(e)
except Exception as e:
    display(e)



print(discriminator_train_data[0][1][0][0:3])
out = discriminator(*discriminator_train_data[0][1][0][0:3])
print(out)

In [ ]:
print(discriminator.eval())
model_parameters = filter(lambda p: p.requires_grad, discriminator.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

## train

In [ ]:
import torch.optim as optim
# discriminate 2 things: which image is the input and which is the output
criterion = nn.BCELoss()
optimizer = optim.Adam(discriminator.parameters(), lr=0.0001)
def train_discriminator(epoch=1, batch_size = 500):
    for epoch in range(epoch):
        running_loss = 0.0
        print("epoch", epoch+1)
        i = 0
        for task in discriminator_train_data:
            for sub_epoch in range(25): # learning for each different task
                for t_inputs, t_outputs, target in task:
                    # zero the parameter gradients
                    optimizer.zero_grad()
                    # get output
                    outputs = discriminator(t_inputs, t_outputs, target)
                    # calculate loss
                    loss = criterion(outputs, i_label)
                    loss.backward()
                    # backprop
                    optimizer.step()
                    # print statistics
                    running_loss += loss.item()
                    i += 1
                    if (i % batch_size) == (batch_size - 1):    # print every 2000 mini-batches
                        print(outputs, i_label)
                        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / batch_size:.3f}')
                        running_loss = 0.0
        torch.save(discriminator.state_dict(), f'/kaggle/working/{discriminator.__class__.__name__}.pth')

train_discriminator()




In [ ]:
# real training
# train_discriminator(50)